<strong><b><font size="5">Análise Estatística e Modelagem Preditiva de Séries Temporais - Em Python</font></b></strong>

<strong><b><font size="5">Projeto Final do Curso - Parte 1</font></b></strong>

<strong><b><font size="4">Qual o Efeito da Legalização da Maconha na Taxa de Criminalidade ao Longo do Tempo?</font></b></strong>

### Carregando os Pacotes Usados Neste Jupyter Notebook

In [1]:
# Comando para atualizar um pacote no terminal ou prompt de comando:
# pip install -U {nome_pacote}

# Para instalar a versão exata de um pacote:
# pip install {nome_pacote}=={versão_desejada}

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook, se necessário.

In [2]:
# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

Os pacotes abaixo serão os pacotes que iremos utilizar para extrair os dados sobre lojas que vendem maconha legalizada.

In [1]:
# Instala o pacote yelp
!pip install -q yelp

In [2]:
# Instala o pacote yelpapi
!pip install -q yelpapi

In [129]:
# As novas versões do Pandas e Matplotlib trazem diversas mensagens de aviso ao desenvolvedor. Vamos desativar isso.
import sys
import warnings
import matplotlib.cbook
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=matplotlib.cbook.mplDeprecation)

# Imports para manipulação de dados
import re
import json
import time
import numpy as np
import pandas as pd
from yelp.client import Client
from yelpapi import YelpAPI

In [130]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark --iversions

json       2.0.9
re         2.2.1
pandas     1.0.4
numpy      1.18.4
matplotlib 3.2.1



### Extração e Limpeza dos Dados - Dataset 1

Vamos coletar os dados de três fontes diferentes e vamos começar pelo <a href="https://www.yelp.com/">Yelp</a>.

O Yelp é serviço que contém avaliações (reviews) e informações sobre empresas em várias regiões do mundo. 

Objetivo: Buscar as lojas de venda de maconha legalizada (chamadas dispensários) na região de Los Angeles, e então, vamos cruzar esses dados com a taxa de crimes e prisões nas mesmas regiões, além de dados sobre escolas. 

###### Observação importante:

Para usar o Yelp, você precisa instalar os pacotes yelp e yelpapi (Feito acima). Depois disso, você deve acessar os endereços abaixo, criar sua conta Yelp e então criar sua app para obter a API de acesso.

Aqui tem os detalhes da API: https://www.yelp.com/developers/documentation/v3/get_started

E aqui o cadastro da app: https://www.yelp.com/developers/documentation/v3/authentication

Após criar sua conta, será necessário a confirmação por e-mail antes de criar o app. Assim que o app é criada, é gerada uma API (uma chave de acesso), coloque a API na célula abaixo onde está 'Minha_api_key'.

###### Não é possível seguir adiante se você não criar sua API. 

###### Existe um limite de 5000 chamadas diárias da API na conta gratuita. Se ultrapassar o limite diário, sua conta pode ser bloqueada. (Informação obtida na própria API).

In [131]:
# Aqui você deve incluir sua API Yelp
minha_api = "Minha_api_Key"

In [132]:
# Com a API definida, criamos um cliente de acesso
cliente_acesso = Client(minha_api)

Função para formatar o arquivo retornado com as consultas da API e gerar um arquivo de log da execução (histórico).

In [133]:
def formata_arquivo(file_path, 
                    logfile = './dados/arquivo_log.txt',  
                    file_description = None): 
   
    # Com o nome completo do arquivo, aplicamos expressões regulares para a limpeza
    # Aqui ajustamos a extensão do arquivo
    try:
        ext = re.search('(?<!^)(?<!\.)\.(?!\.)', file_path).start() 
    except:
        raise NameError('Digite um caminho de onde o arquivo pode ser encontrado.') 
    
    # Com o nome completo do arquivo, aplicamos expressões regulares para a limpeza
    # Aqui ajustamos timestamp para o nome do arquivo
    try:
        stamp = re.search('(?<!^)(?<!\.)[a-z]+_[a-z]+(?=\.)', file_path).start()
    except:
        raise NameError('Digite um caminho de onde o arquivo pode ser encontrado.') 
        
    # Formata o nome do arquivo adicionando o timestamp
    formatted_name = f'{file_path[:stamp]}{round(time.time())}_{file_path[stamp:]}' 
    
    # Se não tiver descrição no arquivo, adicionamos uma
    if not file_description:
        file_description = f'Arquivo gerado em: {time.asctime(time.gmtime(round(time.time())))}'
        
    # Abrimos o arquivo de log e gravamos o arquivo de dados formatado e a descrição
    with open(logfile, 'a+') as f:
        f.write(f'{formatted_name}: {file_description}\n')
        
    # Retornamos o arquivo de dados formatado e a descrição
    return formatted_name, file_description

Função de busca das lojas que vendem maconha legalizada em Los Angeles

In [148]:
# n_exec : numero de execuções que será realizada
# size: quantos registros serão retornados para cada busca dentro da API
def busca_yelp(category, location, offset_number = 0, n_exec = 20, size = 50):
    
    # API
    yelp_api = YelpAPI(minha_api)
    
    # Registro do último resultado
    last_result = round(time.time())
    
    # Lista para gravar os resultados
    resultados = []
        
    # Inicializa o número de loops
    loops = 0
    
    # Inicializa o número de execuções
    run = 1
    
    # Inicializa o offset
    offset_count = offset_number
   
    # Loop para retonar os dados
    while loops < n_exec:
        
        print(f'Executando a busca: {run}')
        
        # Fazendo a consulta
        posts = yelp_api.search_query(categories = category, 
                                      location = location, 
                                      offset = offset_count, 
                                      limit = size) 
            
        # Posts ligados a business
        resultados.extend(posts['businesses'])
        
        # Incrementa o número de loops
        loops += 1
        
        # Incrementa o offset para avançar na busca durante as iterações
        offset_count += 50
        
        # Aguarda 3 segundos para executar a próxima consulta
        time.sleep(3) 
        
        # Incrementa o número de execuções
        run += 1
       
    # Finalizado o loop, vamos retornar o nome do arquivo formatado e a descrição
    formatted_name, file_description = formata_arquivo(file_path =f'./arquivo_{category}.json')
    
    global formatted_name_return 
    formatted_name_return = formatted_name
    # Abrimos o arquivo formatado e fazemos o dump (gravação) dos resultados da consulta em formato JSON
    with open(formatted_name, 'w+') as f:
        json.dump(resultados, f)
    
    print(f'\nConsulta concluída. Total de lojas encontradas: {len(resultados)} {category} .')
    
    # return print(f'\nEsse foi o último timestamp {round(time.time())}.')

In [149]:
# Executamos a busca com o tipo de estabelecimento e a cidade
busca_yelp('cannabisdispensaries', 'los angeles', n_exec = 1, size = 50)

Executando a busca: 1

Consulta concluída. Total de lojas encontradas: 50 cannabisdispensaries .


In [150]:
# Abre o arquivo JSON para leitura e geração da lista final
with open(f'{formatted_name_return}', 'r') as f:
    lista_lojas_la = json.load(f)

In [151]:
# Vamos visualizar um registro da lista
lista_lojas_la[5]

{'id': '_PVsgnMe_ut8BcQSVhAVgw',
 'alias': 'herb-los-angeles-3',
 'name': 'HERB',
 'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/HSw89Daf0Tmxf7aUAXRC6g/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/herb-los-angeles-3?adjust_creative=nKnicr6HZ6bapdL17PUcuQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=nKnicr6HZ6bapdL17PUcuQ',
 'review_count': 96,
 'categories': [{'alias': 'cannabis_clinics', 'title': 'Cannabis Clinics'},
  {'alias': 'cannabisdispensaries', 'title': 'Cannabis Dispensaries'}],
 'rating': 4.5,
 'coordinates': {'latitude': 34.0434989929199, 'longitude': -118.250205993652},
 'transactions': [],
 'price': '$$',
 'location': {'address1': '',
  'address2': None,
  'address3': '',
  'city': 'Los Angeles',
  'zip_code': '90014',
  'country': 'US',
  'state': 'CA',
  'display_address': ['Los Angeles, CA 90014']},
 'phone': '+18444372213',
 'display_phone': '(844) 437-2213',
 'distance': 6707.647988501463}

Vamos organizar e limpar esses dados.

In [155]:
# Essa função vai organizar os dados da lista de lojas em um dataframe do pandas
def organiza_dados(lista_lojas):    
    # Converte a lista de lojas em dataframe do pandas
    df = pd.DataFrame(lista_lojas)
    
    # Lista com os nomes de colunas que nos interessam
    col_list = ['name', 'is_closed', 'url', 'rating', 'coordinates', 'location', 'price', 'review_count']
    
    # Define a lista de colunas acima como nome de cada coluna
    df = df[col_list]

    return df

In [156]:
# Aplica a função e cria nosso dataframe
df_lojas = organiza_dados(lista_lojas_la)

In [159]:
# Shape
df_lojas.shape

(50, 8)

In [160]:
# Visualizamos uma amostra de dados
df_lojas.head()

,name,is_closed,url,rating,coordinates,location,price,review_count
0,California Caregivers Alliance,False,https://www.yelp.com/biz/california-caregivers...,4.5,"{'latitude': 34.08235, 'longitude': -118.272037}","{'address1': '2815 W Sunset Blvd', 'address2':...",NaN,252
1,Herbarium,False,https://www.yelp.com/biz/herbarium-west-hollyw...,4.5,"{'latitude': 34.08853, 'longitude': -118.3446}","{'address1': '979 N La Brea Ave', 'address2': ...",$$,263
2,Kushfly,False,https://www.yelp.com/biz/kushfly-los-angeles-5...,4.0,"{'latitude': 34.127617, 'longitude': -118.34671}","{'address1': '', 'address2': None, 'address3':...",$$,248
3,Green Earth Collective,False,https://www.yelp.com/biz/green-earth-collectiv...,4.5,"{'latitude': 34.12247, 'longitude': -118.21067}","{'address1': '4801 York Blvd', 'address2': '',...",$$,195
4,MedMen Venice Beach - Abbot Kinney,False,https://www.yelp.com/biz/medmen-venice-beach-a...,4.5,"{'latitude': 33.99088, 'longitude': -118.46783}","{'address1': '1310 Abbot Kinney Blvd', 'addres...",$$,173


Vamos extrair a lista de latitudes e longitudes de cada loja, pois usaremos geolocalização mais tarde em outra etapa.

In [165]:
# Realiza o acesso através do index e navega na coluna coordinates propriedade latitude

# Extrai a lista de latitudes
lista_latitude = [lista_lojas_la[i]['coordinates']['latitude'] for i in range(len(lista_lojas_la))]

# Extrai a lista de longitudes
lista_longitude = [lista_lojas_la[i]['coordinates']['longitude'] for i in range(len(lista_lojas_la))]

In [166]:
# Adiciona latitude e longitude ao dataframe df_lojas
df_lojas['latitude'] = lista_latitude
df_lojas['longitude'] = lista_longitude

In [167]:
# Visualizamos uma amostra de dados
df_lojas.head()

,name,is_closed,url,rating,coordinates,location,price,review_count,latitude,longitude
0,California Caregivers Alliance,False,https://www.yelp.com/biz/california-caregivers...,4.5,"{'latitude': 34.08235, 'longitude': -118.272037}","{'address1': '2815 W Sunset Blvd', 'address2':...",NaN,252,34.082350,-118.272037
1,Herbarium,False,https://www.yelp.com/biz/herbarium-west-hollyw...,4.5,"{'latitude': 34.08853, 'longitude': -118.3446}","{'address1': '979 N La Brea Ave', 'address2': ...",$$,263,34.088530,-118.344600
2,Kushfly,False,https://www.yelp.com/biz/kushfly-los-angeles-5...,4.0,"{'latitude': 34.127617, 'longitude': -118.34671}","{'address1': '', 'address2': None, 'address3':...",$$,248,34.127617,-118.346710
3,Green Earth Collective,False,https://www.yelp.com/biz/green-earth-collectiv...,4.5,"{'latitude': 34.12247, 'longitude': -118.21067}","{'address1': '4801 York Blvd', 'address2': '',...",$$,195,34.122470,-118.210670
4,MedMen Venice Beach - Abbot Kinney,False,https://www.yelp.com/biz/medmen-venice-beach-a...,4.5,"{'latitude': 33.99088, 'longitude': -118.46783}","{'address1': '1310 Abbot Kinney Blvd', 'addres...",$$,173,33.990880,-118.467830


In [173]:
# Como já temos os dados de latitude e longitude, a coluna location com o endereço completo e a coluna coordinates 
# podem ser removidas do dataframe.
df_lojas.drop('location', axis = 1, inplace = True)

# E removemos a coluna com as coordenadas
df_lojas.drop(labels = ['coordinates'], axis = 1, inplace = True)

In [174]:
# Criando uma nova coluna location contendo uma tupla com latitude e longitude
df_lojas['location'] = list(zip(df_lojas['latitude'], df_lojas['longitude']))

In [176]:
# Conferimos o shape
df_lojas.shape

(50, 9)

In [175]:
# Visualizamos uma amostra de dados
df_lojas.head()

,name,is_closed,url,rating,price,review_count,latitude,longitude,location
0,California Caregivers Alliance,False,https://www.yelp.com/biz/california-caregivers...,4.5,NaN,252,34.082350,-118.272037,"(34.08235, -118.272037)"
1,Herbarium,False,https://www.yelp.com/biz/herbarium-west-hollyw...,4.5,$$,263,34.088530,-118.344600,"(34.08853, -118.3446)"
2,Kushfly,False,https://www.yelp.com/biz/kushfly-los-angeles-5...,4.0,$$,248,34.127617,-118.346710,"(34.127617, -118.34671)"
3,Green Earth Collective,False,https://www.yelp.com/biz/green-earth-collectiv...,4.5,$$,195,34.122470,-118.210670,"(34.12247, -118.21067)"
4,MedMen Venice Beach - Abbot Kinney,False,https://www.yelp.com/biz/medmen-venice-beach-a...,4.5,$$,173,33.990880,-118.467830,"(33.99088, -118.46783)"


##### Finalizamos a limpeza do nosso primeiro dataset. Vamos salvá-lo em disco para ser possível carregar depois nas etapas seguintes.

In [178]:
# Salva o dataframe de lojas em um arquivo csv
df_lojas.to_csv('dados/df_lojas.csv')

### Extração e Limpeza dos Dados - Dataset 2

O segundo dataset é sobre os crimes que ocorreram em Los Angeles entre 2010 e 2019. O dataset já está disponivel mas caso queira fazer o download direto da fonte ou estudar o dicionário de dados, siga os passos abaixo:

- 1- Acesse o endereço: https://data.lacity.org/

- 2- Descendo um pouco na página, use a caixa de busca de datasets e pesquise por "Crime Data".

- 3- O que você precisa deve ser a primeira opção: "Crime Data from 2010 to 2019". Clique no nome do dataset.

- 4- Na página seguinte clique no botão azul "View Data". Clique então em "Export" - "Download" - "CSV".

- 5- Coloque o arquivo na pasta dados no mesmo diretório onde está este Jupyter Notebook. O arquivo tem mais de 2 milhões de registros.

In [187]:
# Vamos carregar o arquivo com os crimes em LA entre 2010 e 2019
df_crimes = pd.read_csv('dados/Crime_Data_from_2010_to_2019.csv')

In [188]:
# Shape
df_crimes.shape

(2114238, 28)

In [189]:
# Visualizamos uma amostra de dados
df_crimes.head()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,1307355,02/20/2010 12:00:00 AM,02/20/2010 12:00:00 AM,1350,13,Newton,1385,2,900,VIOLATION OF COURT ORDER,...,AA,Adult Arrest,900.0,NaN,NaN,NaN,300 E GAGE AV,NaN,33.9825,-118.2695
1,11401303,09/13/2010 12:00:00 AM,09/12/2010 12:00:00 AM,45,14,Pacific,1485,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",...,IC,Invest Cont,740.0,NaN,NaN,NaN,SEPULVEDA BL,MANCHESTER AV,33.9599,-118.3962
2,70309629,08/09/2010 12:00:00 AM,08/09/2010 12:00:00 AM,1515,13,Newton,1324,2,946,OTHER MISCELLANEOUS CRIME,...,IC,Invest Cont,946.0,NaN,NaN,NaN,1300 E 21ST ST,NaN,34.0224,-118.2524
3,90631215,01/05/2010 12:00:00 AM,01/05/2010 12:00:00 AM,150,6,Hollywood,646,2,900,VIOLATION OF COURT ORDER,...,IC,Invest Cont,900.0,998.0,NaN,NaN,CAHUENGA BL,HOLLYWOOD BL,34.1016,-118.3295
4,100100501,01/03/2010 12:00:00 AM,01/02/2010 12:00:00 AM,2100,1,Central,176,1,122,"RAPE, ATTEMPTED",...,IC,Invest Cont,122.0,NaN,NaN,NaN,8TH ST,SAN PEDRO ST,34.0387,-118.2488


In [190]:
# Substituindo nomes de colunas com espaço por underline, para poder usar os nomes das colunas como filtro de indexação.
df_crimes.columns = [column.lower().replace(' ', '_') for column in df_crimes.columns]

In [191]:
# Verificando se existem valores nulos no dataset
df_crimes.isnull().sum()

dr_no                   0
date_rptd               0
date_occ                0
time_occ                0
area_                   0
area_name               0
rpt_dist_no             0
part_1-2                0
crm_cd                  0
crm_cd_desc             0
mocodes            227959
vict_age                0
vict_sex           196652
vict_descent       196699
premis_cd              53
premis_desc           187
weapon_used_cd    1404139
weapon_desc       1404140
status                  3
status_desc             0
crm_cd_1               10
crm_cd_2          1975051
crm_cd_3          2110747
crm_cd_4          2114134
location                0
cross_street      1758896
lat                     0
lon                     0
dtype: int64

Vamos fazer uma limpeza geral nos dados removendo colunas que não serão utilizadas.

Removendo colunas que não serão necessárias. 
###### É possível obter o dicionário de dados na página para download do arquivo para informações sobre cada coluna.

In [193]:
df_crimes.drop(labels = ['crm_cd', 'crm_cd_1', 'crm_cd_2', 'crm_cd_3', 'crm_cd_4',
                         'premis_cd', 'premis_desc', 'vict_descent', 'vict_sex', 'status',
                         'dr_no', 'area_', 'date_rptd', 'rpt_dist_no', 'part_1-2',
                         'mocodes', 'cross_street', 'weapon_used_cd',
                         'status_desc', 'time_occ', 'vict_age'
                        ], 
               axis = 1, 
               inplace = True
              )

In [194]:
# A coluna weapon_desc vamos manter e preencher valores NA com 'desconhecido'
df_crimes.weapon_desc.fillna('desconhecido', inplace = True)

In [195]:
# Vamos checar se ainda sobrou valor nulo
df_crimes.isnull().sum()

date_occ       0
area_name      0
crm_cd_desc    0
weapon_desc    0
location       0
lat            0
lon            0
dtype: int64

In [202]:
# Tipos de dados
df_crimes.dtypes

date_occ        object
area_name       object
crm_cd_desc     object
weapon_desc     object
location        object
lat            float64
lon            float64
dtype: object

Variáveis ajustadas e nenhum dado missing.

In [196]:
# Visualizamos uma amostra de dados
df_crimes.head()

,date_occ,area_name,crm_cd_desc,weapon_desc,location,lat,lon
0,02/20/2010 12:00:00 AM,Newton,VIOLATION OF COURT ORDER,desconhecido,300 E GAGE AV,33.9825,-118.2695
1,09/12/2010 12:00:00 AM,Pacific,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",desconhecido,SEPULVEDA BL,33.9599,-118.3962
2,08/09/2010 12:00:00 AM,Newton,OTHER MISCELLANEOUS CRIME,desconhecido,1300 E 21ST ST,34.0224,-118.2524
3,01/05/2010 12:00:00 AM,Hollywood,VIOLATION OF COURT ORDER,HAND GUN,CAHUENGA BL,34.1016,-118.3295
4,01/02/2010 12:00:00 AM,Central,"RAPE, ATTEMPTED","STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",8TH ST,34.0387,-118.2488


In [197]:
# Salva o dataframe de crimes em um arquivo csv
df_crimes.to_csv('dados/df_crimes.csv')

### Extração e Limpeza dos Dados - Dataset 3

O terceiro dataset é referente as prisões em LA. Os dados podem ser obtidos com o mesmo procedimento descrito no Dataset 2, com apenas uma mudança:

- 1- Acesse o endereço: https://data.lacity.org/

- 2- Descendo um pouco na página, use a caixa de busca de datasets e pesquise por "Arrests".

- 3- O que você precisa deve ser a primeira opção: "Arrest Data from 2010 to Present". Clique no nome do dataset.

- 4- Na página seguinte clique no botão azul "View Data". Clique então em "Export" - "Download" - "CSV".

- 5- Coloque o arquivo na pasta dados no mesmo diretório onde está este Jupyter Notebook. O arquivo tem mais de 1 milhão de registros.

O arquivo csv contém aproximadamente 1,2 milhão de prisões. O conjunto de dados é fornecido pelo Departamento de Polícia de Los Angeles (LAPD). Cada linha é o registro de um detido e cada coluna contém detalhes sobre o incidente da prisão. Para os fins deste projeto, examinaremos apenas as detenções relacionadas à maconha aplicando um filtro à `Charge Description` se a coluna contiver a palavra `marijuana` (maconha em inglês).

Calcularemos então a distância entre cada prisão e dispensário de maconha (loja legalizada) e examinaremos o número de prisões por maconha a menos de 800 quilômetros de um dispensário a cada ano (faremos isso na Análise Exploratória).

#### Não é necessário o Download dos dados, já está disponível.

In [198]:
# Vamos carregar o arquivo com as prisoes em LA de 2010 ao momento presente (Data que foi extraído)
df_prisoes = pd.read_csv('dados/Arrest_Data_from_2010_to_Present.csv')

In [199]:
# Shape
df_prisoes.shape

(1350103, 17)

In [200]:
# Visualizamos uma amostra de dados
df_prisoes.head()

,Report ID,Arrest Date,Time,Area ID,Area Name,Reporting District,Age,Sex Code,Descent Code,Charge Group Code,Charge Group Description,Arrest Type Code,Charge,Charge Description,Address,Cross Street,Location
0,200110044,03/27/2020,2125.0,1,Central,142,20,F,W,13.0,Prostitution/Allied,M,647(B)PC,PROSTITUTION,400 S FIGUEROA ST,NaN,"(34.0535, -118.256)"
1,200110045,04/01/2020,1800.0,1,Central,166,44,F,H,18.0,Drunkeness,M,41.27(C)LAM,DRINKING IN PUBLIC***,WINSTON ST,SAN PEDRO ST,"(34.0421, -118.2469)"
2,200110271,03/08/2020,1545.0,1,Central,166,61,M,B,18.0,Drunkeness,M,41.27(C)LAM,DRINKING IN PUBLIC***,6TH,SAN JULIAN,"(34.0428, -118.2461)"
3,201908605,04/08/2020,1530.0,19,Mission,1987,33,M,H,NaN,NaN,M,25620(A)BP,NaN,WOODMAN,CHASE,"(34.228, -118.4344)"
4,191811472,05/03/2019,1700.0,18,Southeast,1802,23,F,B,NaN,NaN,M,653.22 PC,NaN,91ST,FIGUEROA,"(33.9543, -118.2827)"


In [201]:
# Tipos de dados
df_prisoes.dtypes

Report ID                     int64
Arrest Date                  object
Time                        float64
Area ID                       int64
Area Name                    object
Reporting District            int64
Age                           int64
Sex Code                     object
Descent Code                 object
Charge Group Code           float64
Charge Group Description     object
Arrest Type Code             object
Charge                       object
Charge Description           object
Address                      object
Cross Street                 object
Location                     object
dtype: object

In [203]:
# Vamos converter a coluna 'Arrest Date' de string (object) para Datetime, para facilitar a manipulação
df_prisoes['Arrest Date'] = pd.to_datetime(df_prisoes['Arrest Date'])

In [204]:
# Vamos usar expressões regulares para transformar a coluna Location de tupla em lista para facilitar a indexação na hora 
# de separar em 2 novas colunas.
df_prisoes['Location'] = df_prisoes['Location'].map(lambda x: re.sub('[(),°]', '', x)).str.split()

In [205]:
# Vamos criar as variáveis latitude e longitude
df_prisoes['latitude'] = df_prisoes['Location'].map(lambda x: x[0])
df_prisoes['longitude'] = df_prisoes['Location'].map(lambda x: x[1])

In [206]:
# Convertemos latitude e longitude para o tipo float para não perder a precisão
df_prisoes['latitude'] = df_prisoes['latitude'].map(lambda x: float(x))
df_prisoes['longitude'] = df_prisoes['longitude'].map(lambda x: float(x))

In [207]:
# Convertemos os registros da coluna 'Charge Description' em string e minúsculo
df_prisoes['Charge Description'] = df_prisoes['Charge Description'].map(lambda x: str(x))
df_prisoes['Charge Description'] = df_prisoes['Charge Description'].map(lambda x: x.lower())

In [208]:
# Visualizamos uma amostra de dados
df_prisoes.head()

,Report ID,Arrest Date,Time,Area ID,Area Name,Reporting District,Age,Sex Code,Descent Code,Charge Group Code,Charge Group Description,Arrest Type Code,Charge,Charge Description,Address,Cross Street,Location,latitude,longitude
0,200110044,2020-03-27,2125.0,1,Central,142,20,F,W,13.0,Prostitution/Allied,M,647(B)PC,prostitution,400 S FIGUEROA ST,NaN,"[34.0535, -118.256]",34.0535,-118.2560
1,200110045,2020-04-01,1800.0,1,Central,166,44,F,H,18.0,Drunkeness,M,41.27(C)LAM,drinking in public***,WINSTON ST,SAN PEDRO ST,"[34.0421, -118.2469]",34.0421,-118.2469
2,200110271,2020-03-08,1545.0,1,Central,166,61,M,B,18.0,Drunkeness,M,41.27(C)LAM,drinking in public***,6TH,SAN JULIAN,"[34.0428, -118.2461]",34.0428,-118.2461
3,201908605,2020-04-08,1530.0,19,Mission,1987,33,M,H,NaN,NaN,M,25620(A)BP,nan,WOODMAN,CHASE,"[34.228, -118.4344]",34.2280,-118.4344
4,191811472,2019-05-03,1700.0,18,Southeast,1802,23,F,B,NaN,NaN,M,653.22 PC,nan,91ST,FIGUEROA,"[33.9543, -118.2827]",33.9543,-118.2827


In [209]:
# Shape
df_prisoes.shape

(1350103, 19)

Vamos extrair prisões relacionadas à maconha.

**Obs: Maconha tem muitas denominações em inglês, mas nesses datasets está registrada como marijuana**.

In [210]:
# Vamos extrair os tipos de prisões distintos e colocar em uma lista ordenando por quantidade de registros.
lista_prisoes = list(df_prisoes['Charge Description'].value_counts().index.sort_values())

In [215]:
# Vamos colocar em minúsculo para usar os dados no mesmo padrão
lista_prisoes = [x.lower() for x in lista_prisoes]

In [216]:
# Tipos de prisões
lista_prisoes

['290 viol post incarc',
 '2nd degree burglary during earthquake/etc',
 '< age 21 driving veh w/blood-alcohol .01+',
 'abandon dog or cat',
 'abandon/non-support child under 14 yrs',
 'accept wager/hold bets',
 'accepting a wager - hold bets/wagers',
 'access card forgery',
 'access computer and make copies',
 'access computer to defraud/deceive/extort',
 'access/delete computer data',
 'accessory aftr fact-know fel crime/person',
 'acquire access card in 4+ names > 12 mos',
 'acquire access card in 4+names >12 mos',
 'acquire access card known to be lost',
 'acquire access card w/intent to defraud',
 'acquire access card w/intent to use/sell',
 'acqure access card account number',
 'act as vehicle salesman w/o license',
 'act unlawfully as auctioneer',
 'acting as a look-out',
 'ad 4 sale/etc recordg/etc w/o origin w/pr',
 'administer/etc cntl subs to addict',
 'adult school/molest pupils',
 'adult sell/etc cntl sub/narcotic to mi',
 'adult solicit/etc mi:viol cntl sub/narco',
 'adult

In [217]:
# Vamos filtrar somente prisões por marijuana. Vamos usar 'mari' na pesquisa pois pode haver abreviações.
[x for x in lista_prisoes if 'mari' in x]

['attempt - sell/furnish/etc marijuana',
 'attempt to poss < 28.5 grams of marijuana',
 'cultivate >6 marij plants viol envrnt law',
 'cultivating <6 marijuana plants',
 'furnishing marijuana to minor over 14 yrs',
 'give/transport/etc < 28.5 grams marijuana',
 'induce/etc minor to use/sell marijuana',
 'minor poss 28.5+ grams marijuana/school',
 'minor poss < 28.5 grams marijuana/school',
 'poss for sale of marijuana to a minor',
 'poss marijuana for sale w/two/more priors',
 'poss marijuana or concentrated cannabis',
 'poss of more than 28.5 grams of marijuana',
 'poss open cont/packg marij drivr/passnger',
 'poss/sale marij ovr 21 employ per 20/belw',
 'poss/smoke/ingest marij school/daycare/yc',
 'possess 28.5 grams or less of marijuana',
 'possess 28.5 grams or less of marijuana**',
 'possess marijuana for sale',
 'possess marijuana for sale under age 18',
 'possess of marijuana while driving veh',
 'possession marijuana for sale',
 'possession of marijuana in school',
 'sale/offe

In [218]:
# Qual o total de prisões que contém a palavra 'mari'?
len([x for x in lista_prisoes if 'mari' in x])

34

Existem 34 diferentes tipos de prisões por causa de maconha que tratam de cultivar, possuir, vender, fumar em público e lidar com menores.

In [219]:
# Criamos uma lista com as razões de prisão por maconha
prisoes_maconha = [x for x in lista_prisoes if 'mari' in x]

In [220]:
# Criamos uma coluna no dataframe que identifica as prisões se elas estiverem relacionadas à maconha
df_prisoes['marijuana_related'] = df_prisoes['Charge Description'].map(lambda x: x if x in prisoes_maconha else np.NaN)

In [221]:
# Visualizamos uma amostra de dados
df_prisoes.head()

,Report ID,Arrest Date,Time,Area ID,Area Name,Reporting District,Age,Sex Code,Descent Code,Charge Group Code,Charge Group Description,Arrest Type Code,Charge,Charge Description,Address,Cross Street,Location,latitude,longitude,marijuana_related
0,200110044,2020-03-27,2125.0,1,Central,142,20,F,W,13.0,Prostitution/Allied,M,647(B)PC,prostitution,400 S FIGUEROA ST,NaN,"[34.0535, -118.256]",34.0535,-118.2560,NaN
1,200110045,2020-04-01,1800.0,1,Central,166,44,F,H,18.0,Drunkeness,M,41.27(C)LAM,drinking in public***,WINSTON ST,SAN PEDRO ST,"[34.0421, -118.2469]",34.0421,-118.2469,NaN
2,200110271,2020-03-08,1545.0,1,Central,166,61,M,B,18.0,Drunkeness,M,41.27(C)LAM,drinking in public***,6TH,SAN JULIAN,"[34.0428, -118.2461]",34.0428,-118.2461,NaN
3,201908605,2020-04-08,1530.0,19,Mission,1987,33,M,H,NaN,NaN,M,25620(A)BP,nan,WOODMAN,CHASE,"[34.228, -118.4344]",34.2280,-118.4344,NaN
4,191811472,2019-05-03,1700.0,18,Southeast,1802,23,F,B,NaN,NaN,M,653.22 PC,nan,91ST,FIGUEROA,"[33.9543, -118.2827]",33.9543,-118.2827,NaN


Valores nulos foram gerados quando não se trata de uma prisão relacionada a maconha. Vamos contar o que NÃO é valor nulo.

In [222]:
len(df_prisoes[~df_prisoes['marijuana_related'].isnull()])

20673

Temos 20.673 prisões por razões relacionadas à maconha!

In [223]:
# Removendo os registros que não correspondem a prisões relacionadas a maconha.
df_prisoes = df_prisoes[~df_prisoes['marijuana_related'].isnull()]

In [224]:
# Shape
df_prisoes.shape

(20673, 20)

In [225]:
# Visualizamos uma amostra de dados
df_prisoes.head()

,Report ID,Arrest Date,Time,Area ID,Area Name,Reporting District,Age,Sex Code,Descent Code,Charge Group Code,Charge Group Description,Arrest Type Code,Charge,Charge Description,Address,Cross Street,Location,latitude,longitude,marijuana_related
340,5568617,2019-03-09,2015.0,6,Hollywood,646,29,M,O,16.0,Narcotic Drug Laws,F,11359HS,possession marijuana for sale,CHEROKEE,HOLLYWOOD,"[34.1016, -118.335]",34.1016,-118.3350,possession marijuana for sale
342,5568629,2019-03-09,2015.0,6,Hollywood,646,25,M,B,16.0,Narcotic Drug Laws,F,11359HS,possession marijuana for sale,CHEROKEE,HOLLYWOOD,"[34.1016, -118.335]",34.1016,-118.3350,possession marijuana for sale
983,5607178,2019-04-21,2115.0,7,Wilshire,702,42,M,B,16.0,Narcotic Drug Laws,F,11359(D)HS,poss/sale marij ovr 21 employ per 20/belw,MELROSE AV,FAIRFAX AV,"[34.0838, -118.3614]",34.0838,-118.3614,poss/sale marij ovr 21 employ per 20/belw
1249,200608837,2020-03-17,2200.0,6,Hollywood,646,21,F,B,16.0,Narcotic Drug Laws,I,11362.3A1HS,smoke/ingest marijuana in public place,HOLLYWOOD,VINE,"[34.1016, -118.3267]",34.1016,-118.3267,smoke/ingest marijuana in public place
1255,200608903,2020-03-19,2320.0,6,Hollywood,645,25,M,W,16.0,Narcotic Drug Laws,I,11362.3A1HS,smoke/ingest marijuana in public place,HOLLYWOOD,HIGHLAND,"[34.1016, -118.3387]",34.1016,-118.3387,smoke/ingest marijuana in public place


In [226]:
# Salva o dataframe de prisões em um arquivo csv
df_prisoes.to_csv('dados/df_prisoes.csv')

### Extração e Limpeza dos Dados - Dataset 4

Iremos calcular a distância entre todas as prisões por maconha e as escolas do Distrito Escolar Unificado de Los Angeles (LAUSD), examinando o número de prisões por maconha a menos de 800 quilômetros de uma escola a cada ano.

Os dados estão disponíveis neste endereço:

http://www.lausd.k12.ca.us/lausd/offices/bulletins/

E este é o arquivo que precisamos:

http://www.lausd.k12.ca.us/lausd/offices/bulletins/lausdk12.tab

Dados já disponiveis.

In [227]:
# Carregamos o arquivo com dados de escolas de LA
df_escolas = pd.read_csv('dados/lausdk12.tab', sep = '\t')

In [228]:
# Shape
df_escolas.shape

(1386, 14)

In [229]:
# Visualizamos uma amostra de dados
df_escolas.head()

,Cost Center Code,Legacy Code,School,Address,City,State,Zip Code,Telephone,Fax,Grades,Calendar,Local District,Board District,File Build Date
0,1585701,5857,107th Street Elementary,147 E 107th St,Los Angeles,CA,90003,323-756-8137,323-779-6942,K- 5,1 Trk,S,7-Dr Richard A Vladovic,2020-05-14
1,1585702,5858,107th Street Elementary Science/Tech/Eng/Arts/...,147 E 107th St,Los Angeles,CA,90003,323-756-8137,323-779-6942,1- 5,1 Trk,S,7-Dr Richard A Vladovic,2020-05-14
2,1583601,5836,109th Street Elementary,10915 S Mc Kinley Ave,Los Angeles,CA,90059,323-756-9206,323-755-2307,K- 5,1 Trk,S,7-Dr Richard A Vladovic,2020-05-14
3,1708201,7082,10th Street Elementary,1000 Grattan St,Los Angeles,CA,90015,213-380-8990,213-480-6732,1- 5,1 Trk,C,2-Monica Garcia,2020-05-14
4,1588401,5884,112th Street Elementary,1265 E 112th St,Los Angeles,CA,90059,323-567-2108,323-567-2611,K- 5,1 Trk,S,7-Dr Richard A Vladovic,2020-05-14


Vamos combinar as colunas `Address`, `City`, `State`, `CEP` para criar uma variável `complete_address` que será usada para obter as coordenadas de cada escola, que usaremos na geolocalização.

In [230]:
# Combinando dados de endereços para gerar as coordenadas mais tarde
df_escolas['complete_address'] = df_escolas['Address'] +' '+ df_escolas['City'] +' '+ df_escolas['State'] +' '+ df_escolas['Zip Code'].astype(str)

In [234]:
df_escolas.dtypes

Cost Center Code     int64
Legacy Code          int64
School              object
Address             object
City                object
State               object
Zip Code             int64
Telephone           object
Fax                 object
Grades              object
Calendar            object
Local District      object
Board District      object
File Build Date     object
complete_address    object
dtype: object

In [235]:
# Deletando colunas que não serão necessárias na nossa análise.
df_escolas = df_escolas.drop(['Address', 'City', 'State', 
                              'Cost Center Code', 'Legacy Code', 'Telephone', 'Fax', 'Calendar',
                              'File Build Date'], 
                             1)

In [236]:
# Removemos Registros duplicados baseado na localização do endereço completo, se existirem
df_escolas = df_escolas[~df_escolas.duplicated(subset = 'complete_address')].sort_values('complete_address')

In [237]:
# Reset do índice do dataframe, pois registros podem ter sido removidos
df_escolas.reset_index(drop = True, inplace = True)

In [238]:
# Shape
df_escolas.shape

(947, 6)

In [239]:
# Visualizamos uma amostra de dados
df_escolas.head()

,School,Zip Code,Grades,Local District,Board District,complete_address
0,Frank Del Olmo Elementary,90004,K- 5,C,2-Monica Garcia,100 N New Hampshire Ave Los Angeles CA 90004
1,10th Street Elementary,90015,1- 5,C,2-Monica Garcia,1000 Grattan St Los Angeles CA 90015
2,Elementary Community Day School,91311,K- 6,XS,3-Scott M Schmerelson,10001 Jumilla Ave Chatsworth CA 91311
3,Cesar E Chavez Learning Academy - Arts/Theatre...,91340,9-12,NE,6-Kelly Gonez,1001 Arroyo Ave San Fernando CA 91340
4,San Pedro Senior High Gifted STEAM Magnet,90731,9-12,S,7-Dr Richard A Vladovic,1001 W 15th St San Pedro CA 90731


In [240]:
# Salva o dataframe de escolas em um arquivo csv
df_escolas.to_csv('dados/df_escolas.csv')

Neste ponto temos 4 arquivos csv, um para cada dataset após a limpeza. Iremos utilizar esses datasets nos arquivos seguintes.

# Let's Go Part 2